In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil


from numba import jit

colors = [
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'blue','blue','blue',
'red','red','red',
'green','green','green',
'cyan','cyan','cyan',
'orange','orange','orange',
    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue''darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [10]:
import scipy
import scipy.spatial

@jit
def find_spike_match(s1, s2):
    max_dist = 15
    ctr=0
    for s in s1:
        dist= np.min(np.abs(s-s2))
        if dist<=max_dist:
            ctr+=1
    
    return (ctr)

def find_template_match(data_in, 
                        templates_gt,
                        spike_train_yass,
                        spike_train_gt):
      
    template=data_in[0]
    unit = data_in[1]
    print ("UNIT: ", unit)
        
    template = template.T.ravel()
#     plt.plot(template)
#     plt.show()
#     return (_,_,_,_)

    best_result = 0
    best_roll=0 
    best_match = None
    for t in range(templates_gt.shape[0]):
        data2 = templates_gt[t]
        for k in range(-50,50,1):
            data_test = np.roll(data2,k,axis=0).T.ravel()
            result = 1-scipy.spatial.distance.cosine(template, data_test)
                
            if result>best_result:
                best_roll = k
                best_result = result
                best_match = t

    # compute # matches
    idx_yass = np.where(spike_train_yass[:,1]==unit)[0]
    #print ("spike_train_yass units: ", spike_train_yass[:,1])
    times_yass = spike_train_yass[idx_yass,0]
    #print ("times yass: ", times_yass[:20])

    # find match times
    idx_match = np.where(spike_train_gt[:,1]==best_match+1)[0]
    times_match = spike_train_gt[idx_match,0] + 24
    #print ("times match: ", times_match[:20])
    
    # find matches
    tot_matches = find_spike_match(times_yass, times_match)

    # plot figures
    if False:
        plt.plot(template)
        plt.plot(templates_gt[best_match].T.ravel())
        plt.show()
        
    # compute purity, 
    tot_sorted = idx_yass.shape[0]
    tot_ground_truth = times_match.shape[0]
    n_fn = max((tot_ground_truth-tot_matches), 0)
    n_fp = max((tot_sorted-tot_matches),0)
    
    purity =  tot_matches/ (tot_matches+n_fp)
    accuracy = tot_matches / (n_fn+tot_matches+n_fp)
    
    return (best_match, best_result, best_roll, purity, accuracy,
           n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)


def find_match_spikes(unit, spike_train_yass, spike_train_match):
    
    # compute # matches
    idx_yass = np.where(spike_train_yass[:,1]==unit)[0]
    times_yass = spike_train_yass[idx_yass,0]
    #print ("spike_train_yass units ", unit, times_yass.shape) 
    #if unit!=190:
    #    return (_, _, _, _, _, _, _, _)
    #print ("times yass: ", times_yass[:20])

    best_match_test = 0
    accuracy_test = 0
    for best_match in np.unique(spike_train_gt[:,1]):
        # find match times
        idx_match = np.where(spike_train_gt[:,1]==best_match)[0]
        times_match = spike_train_gt[idx_match,0] + 24
        #print ("times match: ", times_match[:20])

        if times_match.shape[0]==0:
            continue
        # find matches
        tot_matches = find_spike_match(times_yass, times_match)

        # compute purity, 
        tot_sorted = idx_yass.shape[0]
        tot_ground_truth = times_match.shape[0]
        n_fn = max((tot_ground_truth-tot_matches), 0)
        n_fp = max((tot_sorted-tot_matches),0)

        purity =  tot_matches/ max((tot_matches+n_fp),1)
        accuracy = tot_matches / max((n_fn+tot_matches+n_fp),1)

        if accuracy>accuracy_test:
            accuracy_test = accuracy
            best_match_test = best_match
            #print ("accuracy_test: ", accuracy_test)
            #print ("best_match_test: ", best_match_test)
    
    # 
    idx_match = np.where(spike_train_gt[:,1]==best_match_test)[0]
    times_match = spike_train_gt[idx_match,0] + 24

    # find matches
    tot_matches = find_spike_match(times_yass, times_match)

    # compute metrics, 
    tot_sorted = times_yass.shape[0]
    tot_ground_truth = times_match.shape[0]
    
    n_fn = max((tot_ground_truth-tot_matches), 0)
    n_fp = max((tot_sorted-tot_matches),0)

    purity =  tot_matches/ max((tot_matches+n_fp),1)
    accuracy = tot_matches / max((n_fn+tot_matches+n_fp),1)
    print ("unit: ", unit, ", accuracy: ", accuracy)
    
#     print (best_match_test, purity, accuracy,
#            n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)
    
    return (best_match_test, purity, accuracy,
           n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)


def plot_results(templates_yass, spike_train_yass, spike_train_gt, 
                 results,
                idx_in,
                plot_text=False):
    
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    print ("len results: ", len(results))

    purity = []
    sizes= []
    accuracy = []
    fp = []
    fn = []
    for k in range(len(results)):
        best_match_test = results[k][0]
        purity.append(results[k][1])
        tot_sorted = results[k][6]
        tot_ground_truth = results[k][7]
        tot_matches = results[k][5]

        #fp.append(results[k][3]/(tot_ground_truth)
        fp.append((tot_matches/tot_sorted))
        fn.append((tot_matches)/(tot_ground_truth))

        accuracy_local = results[k][5]/(results[k][5]+results[k][3]+results[k][4]) 
        accuracy.append(accuracy_local)

        idx = np.where(spike_train_yass[:,1]==k)[0]
        sizes.append(idx.shape[0])

        if ptps_yass[k]>28:
            print ("yass: ", k, ", gt: ", best_match_test, " ptp: ", 
                   ptps_yass[k], " accuracy: ", accuracy_local)

    sizes=np.array(sizes)
    ptps = templates_yass.ptp(1).max(1)[idx_in]

    ax = plt.subplot(2,2,1)
    print (len(ptps), len(purity))
    plt.scatter(ptps, purity, s=sizes/20.,c='blue',alpha=.5)
    
    if plot_text:
        for k in range(len(purity)):
            plt.text(ptps[k], purity[k], str(k),fontsize=10)
    
    text = '\ntot_matches / (tot_maches + fp)'
    plt.ylabel("Purity / Precision (flatiron): "+text,fontsize=10)
    plt.xlabel("PTP (SU)",fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)
    
    ax=plt.subplot(2,2,2)
    plt.scatter(ptps, accuracy, s=sizes/20.,c='red',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\ntot_matches / (tot_maches + fp + fn)'
    plt.ylabel("Accuracy: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    ax=plt.subplot(2,2,3)
    plt.scatter(ptps, fp, s=sizes/20.,c='orange',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\n(spikes_sorted-tot_matches)/spikes_sorted'
    plt.ylabel("FP: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    ax=plt.subplot(2,2,4)
    plt.scatter(ptps, fn, s=sizes/20.,c='green',alpha=.5)
    plt.xlabel("PTP (SU)",fontsize=10)
    text = '\n(ground_truth - tot_matches)/ground_truth'
    plt.ylabel("FN: "+text,fontsize=10)
    plt.ylim(0,1)
    plt.xlim(left=0)

    plt.suptitle("Match algorithm: accuracy metric (no cosine similarity)")
    #ax.tick_params(axis='both', which='major', labelsize=30)
    plt.show()
    # visualize recomputed templates over time;
    
def binary_reader_waveforms(filename, n_channels, n_times, spikes, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    wfs=[]
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2
        
    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
                #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            try:
                fin.seek(s * data_len * n_channels, os.SEEK_SET)
                wfs.append(np.fromfile(
                    fin,
                    dtype=data_type,
                    count=(n_times * n_channels)).reshape(n_times, n_channels))

            except:
                pass
    fin.close()
    return np.array(wfs)

In [3]:
# load ground truth data

root_dir = '/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/'

spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/ground_truth/spike_train_ground_truth.npy')
# templates_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/templates.npy')
# print (templates_gt.shape)
# ptps_gt = templates_gt.ptp(1).max(1)

# load sorted data
templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/templates_0sec.npy')
print (templates_yass.shape)
ptps_yass = templates_yass.ptp(1).max(1)
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/spike_train.npy')

import parmap
# KS2 vs. Yass
if False:
    results = parmap.map(find_template_match, 
                     list(zip(templates_yass,np.arange(templates_yass.shape[0]))),
                     templates_gt,
                     spike_train_yass,
                     spike_train_gt,
                     pm_processes=12)
else:
    if False:
        for unit in range(10):
            find_match_spikes(unit, 
                 spike_train_yass,
                 spike_train_gt)

    else:
        results = parmap.map(find_match_spikes, #(unit, spike_train_yass, spike_train_match):
                 np.arange(templates_yass.shape[0]), 
                 spike_train_yass,
                 spike_train_gt,
                 pm_processes=12)

        
print ("DONE")

np.save(root_dir + '/results.npy', results)

(10, 151, 32)
unit:  0 unit:  5, accuracy: unit:    7, accuracy:  0.9061496099128041.0
 
, accuracy: unit:   41.0
 , accuracy:  0.9602649006622517
unit:  1 , accuracy:  1.0
unit: unit:  3  , accuracy: 6 1.0 
, accuracy:  0.9886520896761694
unit:  2 , accuracy:  0.598408186469585
unit:  8 , accuracy:  1.0
unit:  9 , accuracy:  0.6731969486823856
DONE


In [3]:
# reload old or new data
if True:
    results = np.load('/media/cat/4TBSSD/data/spikingforest/results_original.npy', allow_pickle=True)
    # load sorted data
    templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/templates.npy', allow_pickle=True)
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
    
else:
    results = np.load('/media/cat/4TBSSD/data/spikingforest/results_old.npy', allow_pickle=True)
    # load sorted data
    templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_old/templates.npy')
    print (templates_yass.shape)
    ptps_yass = templates_yass.ptp(1).max(1)
    spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_old/spike_train.npy')


(221, 101, 60)


In [35]:
# Burst matching function;
templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/templates.npy', allow_pickle=True)
print (templates_yass.shape)
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')

# ground truth data

spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
templates_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/templates.npy')

# load original results
results_original = np.load('/media/cat/4TBSSD/data/spikingforest/results_original.npy',allow_pickle=True)

spike_train_yass_merged = spike_train_yass.copy()
merges = []
ctr2=0
for unit1 in range(templates_yass.shape[0]):
    for unit2 in range(templates_yass.shape[0]):
        if unit1==unit2:
            continue
        if unit1>unit2:
            continue

        temp1 = templates_yass[unit1]
        temp2 = templates_yass[unit2]

        temp1_norm = -temp1/np.min(temp1)
        temp2_norm = -temp2/np.min(temp2)

        diff = temp1_norm - temp2_norm
        if np.max(np.abs(diff))<=0.190:
            print (unit1, unit2, " diff: ", np.max(np.abs(diff)))
            #ax=plt.subplot(5,6,ctr2+1)
            merges.append(np.sort([unit1, unit2]))
            temp1 = templates_yass[unit1]
            temp2 = templates_yass[unit2]
            
            # 
            temp1 = -temp1/np.min(temp1)
            temp2 = -temp2/np.min(temp2)
            #plt.plot(temp1-temp2, c='green',alpha=.5)
            #plt.plot(temp2, c='red', alpha=.5)
            #plt.ylim(-0.1,0.1)

            idx = np.where(spike_train_yass_merged[:,1]==unit2)[0]
            spike_train_yass_merged[idx,1]=unit1
            idx2 = np.where(spike_train_yass_merged[:,1]==unit2)[0]
            idx3 = np.where(spike_train_yass_merged[:,1]==unit1)[0]
            print ("units: ", unit1, unit2, " match", " ", idx.shape[0], idx2.shape[0], idx3.shape[0],
                   ", accuracy1: ", round(results_original[unit1][2],3), 
                   ", accuracy2: ", round(results_original[unit2][2],3),
                   ", total: ", round(results_original[unit1][2] + results_original[unit2][2],3))
            
            ctr2+=1
plt.show()

# reorder dta
spike_train_final = np.zeros((0,2),'int32')
ctr=1
for k in np.unique(spike_train_yass_merged[:,1]):
    idx = np.where(spike_train_yass_merged[:,1]==k)[0]
    
    temp = np.zeros((idx.shape[0],2), 'int32')
    
    temp[:,0] = spike_train_yass_merged[idx,0]
    temp[:,1] = ctr
    spike_train_final = np.vstack((spike_train_final, temp))
    
    ctr+=1

print (spike_train_final.shape)
spike_train_yass_merged = spike_train_final.copy()
print (spike_train_yass_merged)

spike_train_yass_merged[:,1]-=1

# load sorted data
np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train_merged.npy',
       spike_train_yass_merged)




(221, 101, 60)
4 207  diff:  0.1671235
units:  4 207  match   8192 0 10997 , accuracy1:  0.318 , accuracy2:  0.254 , total:  0.572
7 8  diff:  0.14306289
units:  7 8  match   441 0 1140 , accuracy1:  0.31 , accuracy2:  0.314 , total:  0.624
15 35  diff:  0.09507002
units:  15 35  match   2975 0 5883 , accuracy1:  0.38 , accuracy2:  0.421 , total:  0.8
18 169  diff:  0.11288212
units:  18 169  match   2832 0 5438 , accuracy1:  0.437 , accuracy2:  0.503 , total:  0.939
20 180  diff:  0.10584998
units:  20 180  match   780 0 1510 , accuracy1:  0.398 , accuracy2:  0.476 , total:  0.874
24 42  diff:  0.09819445
units:  24 42  match   2870 0 5345 , accuracy1:  0.416 , accuracy2:  0.519 , total:  0.935
44 171  diff:  0.08722196
units:  44 171  match   2972 0 5508 , accuracy1:  0.387 , accuracy2:  0.518 , total:  0.905
47 124  diff:  0.1345562
units:  47 124  match   2950 0 5848 , accuracy1:  0.335 , accuracy2:  0.417 , total:  0.753
52 199  diff:  0.08053971
units:  52 199  match   2873 0 549

In [36]:
# fix templates
templates_del = []
for k in merges:
    templates_del.append(np.sort(k)[1])
templates_del = np.unique(templates_del)
templates_merged = np.delete(templates_yass,templates_del,axis=0)
print ("tempaltes merged: ", templates_merged.shape)

np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates_merged.npy',
       templates_merged)



tempaltes merged:  (197, 101, 60)


In [37]:
import parmap
print (templates_merged.shape)
ptps = templates_merged.ptp(1).max(1)
idx = np.argsort(ptps)[::-1]

idx_in = idx

idx_in = np.arange(templates_merged.shape[0])
print ("testing units: ", idx_in, ptps[idx_in])

results_merged = parmap.map(find_match_spikes, 
                            #np.arange(templates_merged.shape[0]), 
                            idx_in, 
                            spike_train_yass_merged,
#                             np.arange(templates_yass.shape[0]),
#                             spike_train_yass,
                            spike_train_gt,
                            pm_processes=12)
print ("DONE")
np.save('/media/cat/4TBSSD/data/spikingforest/results_merged.npy', results_merged)


(197, 101, 60)
testing units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196] [ 6.0228677 13.394905  11.293156  10.928486   8.945223   3.7195044
 14.2912655  9.407461   3.8813028 23.122099  13.687571  11.72106
  3.6515245  5.1513267  7.724716   5.

In [16]:
# ptps = templates_yass.ptp(1).max(1)
# idx = np.argsort(ptps)[::-1]

# idx_in = idx#[:3]

In [19]:
data = np.fromfile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin','int16')
n_chans = 60
data_2D = data.reshape(-1,n_chans)
print (data_2D.shape)

(19200000, 60)


In [20]:
templates = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates.npy')
print (templates.shape)
max_chans= templates.ptp(1).argmax(1)
ptps = templates.ptp(1).max(1)

(217, 101, 60)


In [21]:
# reloat template 

#fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/standardized.bin'
#data_type = 'float32'
fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin'
data_type = 'int16'

n_channels = 60
n_times = 151

#spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
#spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
    
unit_yass = 199

# load yass waveforms
idx = np.where(spike_train_yass[:,1]==unit_yass)[0]
spikes = np.int32(spike_train_yass[idx,0])-n_times//2
wfs_yass = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_yass.shape)

template_yass = np.mean(wfs_yass,0)
print (template_yass.shape)

(2873, 151, 60)
(151, 60)


In [59]:
# Configure kachery to download data from the public database
from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka
ka.set_config(fr='default_readonly')

# bionet
#recording_path = 'sha1dir://abc900f5cd62436e7c89d914c9f36dcd7fcca0e7.synth_bionet/bionet_static/static_8x_A_2A'

# 32 chan
#recording_path = 'sha1dir://10b2e53b6b3aa0731b763db42daa692c4e1564b0.synth_mearec_neuronexus/datasets_noise10_K20_C32/001_synth'

# janelia
recording_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/static_siprobe/rec_16c_1200s_11'


# LOAD DATA
print ("...loading recording...")
recording = AutoRecordingExtractor(recording_path, download=True)
print ("finished loading recording")

#np.save('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/recording_object', recording)
#sorting_true_path = os.path.join(recording_path,'firings_true.mda')
#sorting = AutoSortingExtractor(sorting_true_path, samplerate=recording.get_sampling_frequency())

...loading recording...
finished loading recording


In [48]:
# np.save('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C
#         32/001_SYNTH/recording_object', recording)


In [60]:
import spiketoolkit as st
import spikeextractors as se

# channel_ids = np.ones(32,'int32')
# recording = se.SubRecordingExtractor(parent_recording=recording, channel_ids=channel_ids)

# load into memory
print('Loading recording into RAM...')
recording = se.NumpyRecordingExtractor(timeseries=recording.get_traces(), 
                                       sampling_frequency=recording.get_sampling_frequency())
print ("DONE")
#recording = st.preprocessing.bandpass_filter(recording=recording, freq_min=300, freq_max=6000)

Loading recording into RAM...
DONE


In [61]:
#print (recording.shape)

AttributeError: 'NumpyRecordingExtractor' object has no attribute 'shape'

In [63]:
# COMPUTE SPIKING FORESET VERSION OF SNR
# filter recording for 10 sec of data on each channel
recording_filtered = st.preprocessing.bandpass_filter(recording=recording, freq_min=300, freq_max=6000)
    
# M=len(channel_ids)
samplerate = int(recording_filtered.get_sampling_frequency())

# grab 1 sec of filtered data;
X = recording_filtered.get_traces(start_frame=samplerate * 1, end_frame=samplerate * 2)
ret = []
n_chan = 16
channel_ids = np.arange(n_chan)
for ii in range(len(channel_ids)):
    # noise_level=np.std(X[ii,:])
    noise_level = np.median(np.abs(X[ii, :])) / 0.6745  # median absolute deviation (MAD)
    ret.append(noise_level)

print ("SNR of each channel:",  ret)
print (len(ret))
#np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/snr.npy',ret)


SNR of each channel: [2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531, 2.965159377316531]
16


In [66]:
print (X.shape)
for k in range(16):
    plt.plot(X[k,:10000]+k*20,c='black')
plt.show()

(16, 30000)


In [13]:
print (X.shape)
print (type(X[0][0]))

(60, 20000)
<class 'numpy.int16'>


In [67]:
recording_filtered.get_traces().T.tofile('/media/cat/4TBSSD/data/spikingforest/hybrid_janelia/static_siprobe/rec_16c_1200s_11/data_filtered.bin')


In [17]:
data1 = np.fromfile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order_filtered_int16.bin',
                   'int16')
print (data1.shape)
data1_2D = data1.reshape(-1,60)

for k in range(60):
    plt.plot(data1_2D[:10000,k]+k*5000,c='black')
plt.show()


(1152000000,)


In [54]:
# Make templates as per spikingforest rules

spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/spike_train.npy')
unit_ids = np.unique(spike_train_yass[:,1])
#print (unit_ids)
data_type = 'int16'
data_type = 'float32'
fname = '/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/c9a62ee5ae1f71097a17acb9b7aca3457161c76e_float32_filtered.bin'
n_channels = n_chan
n_times = 101

templates_sf = []
for unit in unit_ids:
    # print('Unit {} of {}'.format(unit,len(unit_ids)))
    #waveforms = _get_random_spike_waveforms(recording=recording, sorting=sorting, unit=unit, snippet_len=snippet_len, max_num=max_num, channels=None)
    idx = np.where(spike_train_yass[:,1]==unit)[0]
    spikes= spike_train_yass[idx,0]-n_times//2-30
    idx2 = np.where(spikes<=(600*30000-100))[0]
    spikes = spikes[idx2]
    
    waveforms = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
    print (waveforms.shape)
    template = np.median(waveforms, axis=0)
    templates_sf.append(template)

# np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/templates_snr.npy',
#        templates_sf/np.array(ret))

(1104, 101, 32)
(3911, 101, 32)
(8795, 101, 32)
(1274, 101, 32)
(1661, 101, 32)
(4062, 101, 32)
(3613, 101, 32)
(1441, 101, 32)
(1568, 101, 32)
(5762, 101, 32)


In [55]:
plt.plot(templates_sf[0])
plt.show()

In [21]:

# np.save('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/templates_snr.npy',
#        templates_sf/np.array(ret))

In [45]:
#results_merged = np.load('/media/cat/4TBSSD/data/spikingforest/results_merged.npy')
results = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/results.npy')
#print (results)

spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/ground_truth/spike_train_ground_truth.npy')
#templates_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/templates_0sec.npy', allow_pickle=True)
templates_snr = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/templates_snr.npy')
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/spike_train.npy')

#scale templates
#templates_snr /=(85/65.)

# plot original results
fig =plt.figure()
plot_text=True
plot_results(templates_snr, spike_train_yass, spike_train_gt, results, 
             np.arange(len(results)),
            plot_text)

# compute cumulative mean of data
accuracy = results[:,1]
sizes = results[:,6]

ptps = templates_snr.ptp(1).max(1)
idx = np.argsort(ptps)[::-1]

sums_accuracy = []
sums_purity = []
thresh = 8.0#/(65/85.)
for k in range(idx.shape[0]):
    if ptps[idx[k]]<thresh:
        break
    sums_purity.append(results[:,1][idx[k]])
    sums_accuracy.append(results[:,2][idx[k]])

plt.suptitle("Spikingforest Max(Abs(template))/Sigma templates\n"+
             "Mean of Purity (Precision) of units > 8.0 SNR: "+str(round(np.nanmean(sums_purity),2))+"\n"+
             "Mean of Accuracy of units > 8.0 SNR: "+str(round(np.nanmean(sums_accuracy),2)))

if False:
    fig=plt.figure()
    plot_results(templates_yass, spike_train_yass, spike_train_gt, results, np.arange(len(results)))
    plt.suptitle("PTP based measures")

 

(10, 101, 32)
len results:  10
yass:  3 , gt:  1.0  ptp:  30.8779329683016  accuracy:  1.0
yass:  7 , gt:  8.0  ptp:  42.217392180261896  accuracy:  1.0
10 10


In [53]:
# plot specifically identified units:
# best_match_test, purity, accuracy, n_fn, 
# n_fp, tot_matches, tot_sorted, tot_ground_truth)

# yass poor units: 4, 6

def find_matching_spikes_times(spikes_yass, spikes_gt):
    
    tp=[]
    fn=[]
    thresh = 5
    for s in spikes_yass:
        if np.min(np.abs(s-spikes_gt))<5:
            tp.append(s)
        else:
            fp.append(s)
    
    return (np.array(tp), np.array(fp))

fname = '/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/preprocess/standardized.bin'
n_channels = 32
n_times = 101
data_type = 'float32'

# units with problems
yass_units = [4,6,5]
shifts = [15,26, 34]

fig=plt.figure()
for k in range(len(yass_units)):
    yass_unit = yass_units[k]
    gt_unit = int(results[yass_unit][0])
    print ("Yass unit: ", yass_unit, " , gt unit; ", gt_unit)

    # plot templates
    spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/ground_truth/spike_train_ground_truth.npy')
    spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/tmp/output/spike_train.npy')

    # find matches
    idx = np.where(spike_train_yass[:,1]==yass_unit)[0]
    spikes_yass = spike_train_yass[idx,0]

    idx = np.where(spike_train_gt[:,1]==gt_unit)[0]
    spikes_gt = spike_train_gt[idx,0]+shifts[k]

    # 
    #print (spikes_yass)
    #print (spikes_gt)

    tp, fp = find_matching_spikes_times(spikes_yass, spikes_gt)
    #print ("TP: ", tp.shape, "FN: ", fn.shape)

    tp-=n_times//2
    fp-=n_times//2
    
    wfs_tp = binary_reader_waveforms(fname, n_channels, n_times, tp, data_type)
    wfs_fp = binary_reader_waveforms(fname, n_channels, n_times, fp, data_type)

    if True:
        ax = plt.subplot(2,2,k+1)

        template = wfs_tp.mean(0)
        max_chan=template.ptp(0).argmax(0)

        plt.plot(wfs_tp[:,:,max_chan].T,c='black',alpha=.1)

        plt.plot(wfs_fp[:,:,max_chan].T,c='red',alpha=.1)

    plt.title("Yass unit: "+str(yass_units[k])+" , % missing spikes: "+
              str(round(fn.shape[0]/spikes_yass.shape[0],2)*100) ,fontsize=15)
    
plt.suptitle("Analysis of missing spikes in 2 units \nBlack: found spikes    Red: Missing spikes",fontsize=20)
plt.show()

Yass unit:  4  , gt unit;  5
[    2030    12521    16623 ... 17993768 17994133 17998198]
[    2030    12523    16625 ... 17993771 17994135 17998199]
TP:  (1595,) FN:  (66,)
Yass unit:  6  , gt unit;  13
[    5731     9621    11144 ... 17988825 17995891 17998715]
[    5731     9623    11144 ... 17988827 17995894 17998716]
TP:  (3572,) FN:  (41,)
Yass unit:  5  , gt unit;  10
[    1026     1718    18327 ... 17981869 17992229 17995983]
[    1027     1719    18328 ... 17992231 17995985 17999024]
TP:  (3949,) FN:  (113,)


In [56]:
neurons_found = results[:,0]
print ("Neurons found: ", neurons_found)

Neurons found:  [ 3. 12. 16.  1.  5. 10. 13.  8.  2. 17.]


In [81]:
wfs_array=[]
for k in range(20):
    idx = np.where(spike_train_gt[:,1]==k)[0]
    
    spikes = spike_train_gt[idx,0]-n_times//2
    wfs = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
    wfs_array.append(wfs)
    
    

In [100]:
fig=plt.figure()
wfs_array = np.array(wfs_array)
print (wfs_array.shape)
temps = []
for k in range(wfs_array.shape[0]):
    temp = wfs_array[k].mean(0)
    temps.append(temp)

temps = np.array(temps)
print (temps.shape)
ptps = temps.ptp(1).max(1)
idx = np.argsort(ptps)[::-1]
print (ptps[idx])

ctr=1
for k in idx:
    ax=plt.subplot(4,5,ctr)
    wfs = wfs_array[k]
    
    template = wfs.mean(0)
    max_chan= template.ptp(0).argmax(0)
    
    if k in neurons_found:
        clr='black'
        plt.title("Found, chan: "+str(max_chan))
    else:
        clr='red'
        plt.title("Missed, chan: "+str(max_chan))
        
    plt.xticks([])
    plt.plot(wfs[::10,:,max_chan].T,c=clr,alpha=.02)
    plt.plot(template[:,max_chan],linewidth=4, c='blue')
    ctr+=1
plt.suptitle("All 20 ground truth units analysis \rRed: example missed spikes; Black: example found spikes;  Blue: template")
plt.show()

(20,)
(20, 101, 32)
[27.54469   19.125742  10.983813   9.265972   8.8300705  8.824865
  6.8722024  5.7695966  3.907918   3.62894    3.5775702  3.5643072
  3.559717   3.4633448  3.1240668  3.0901875  3.0264156  2.8911786
  2.8578587  2.662494 ]


/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:211: RuntimeWarning: Glyph 13 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:180: RuntimeWarning: Glyph 13 missing from current font.
  font.set_text(s, 0, flags=flags)


In [28]:
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/ground_truth/spike_train_ground_truth.npy')


(197, 8)
0.8814901310238522


/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice.


In [65]:
# plot merged results
#fig =plt.figure()
#spike_train_yass_merged = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train_merged.npy')
##templates_merged = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/templates_merged.npy')
#idx_in = np.arange(templates_merged.shape[0])
#print ("res", results_merged.shape)
plot_results(templates_yass, spike_train_yass, spike_train_gt, results, np.arange(len(results)))

(10, 151, 32)
len results:  10
10 10


In [10]:
idx = np.where(np.logical_and(ptps>28.5, ptps<29.5))
print (idx)
selected_unit = idx[0]

print (len(results))

print (results[selected_unit])

(array([199]),)
221
[[2.17000000e+02 9.70414201e-01 5.04615385e-01 2.65200000e+03
  8.50000000e+01 2.78800000e+03 2.87300000e+03 5.44000000e+03]]


In [74]:
# find the split off match between two units
for k in range(len(results)):
    if results[k][0]==217:
        print (k)


52
199


In [25]:
print (templates_yass.shape)
temp1 = templates_yass[52]
temp2 = templates_yass[199]
ax1 = plt.subplot(2,2,1)

plt.plot(temp1,c='black')
plt.plot(temp2, c='red')
plt.title("Raw templates")

temp1_norm = -temp1/np.min(temp1)
temp2_norm = -temp2/np.min(temp2)
ax1 = plt.subplot(2,2,2)
plt.plot(temp1_norm, c='black')
plt.plot(temp2_norm, c='red')
plt.title("Normalized templates")

ax3 = plt.subplot(2,2,3)
diff = temp1_norm - temp2_norm
plt.plot(diff,c='green')
plt.ylim(-.1,.1)
plt.title("Diff between normalized templates")
plt.ylabel("Normalized value")
plt.suptitle("Bionet simulation; analysis of bursting unit 217; \nyass oversplit into units 52 (black) and 199 (red)")

ax3 = plt.subplot(2,2,4)
diff = (temp1_norm - temp2_norm)* np.min(temp1)
plt.plot(diff,c='green')
#plt.ylim(-.1,.1)
plt.title("Diff between normalized templates")
plt.ylabel("SU values")
plt.suptitle("Bionet simulation; analysis of bursting unit 217; \nyass oversplit into units 52 (black) and 199 (red)")



plt.show()

(221, 101, 60)


In [40]:
#     return (best_match_test, purity, accuracy,
#            n_fn, n_fp, tot_matches, tot_sorted, tot_ground_truth)

# thresholded purity metric
results_merged = np.load('/media/cat/4TBSSD/data/spikingforest/results_merged.npy')

print (results_merged.shape)
accuracy = results_merged[:,2]
sizes = results_merged[:,6]
clr_out = 'red'
#clr_out = 'blue'
data=np.array(accuracy)


ax=plt.subplot(1,1,1)
plt.scatter(ptps, data, s=sizes/20.,c=clr_out,alpha=.5)
#ax=plt.subplot(1,2,2)
idx = np.argsort(ptps)[::-1]


cum_purity = []
for k in range(idx.shape[0]):
    cum_purity.append(data[idx][:k].mean(0))
    
plt.plot(ptps[idx],cum_purity,linewidth=6, c='black')
plt.ylim(0,1)
plt.plot([8,8],[0,1],'r--',linewidth=3,c='black')
plt.title("accuracy")
plt.show()


(197, 8)


/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice.
/home/cat/anaconda3/envs/spikingforest/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
unit = 100
plt.plot(templates_yass[unit])
plt.title("Unit: "+str(unit))
plt.show()

In [21]:
# units=np.arange(5)
# units = [good_units[3]]

# for unit in units:
#     fig = plt.figure()
#     ax = plt.subplot(131)
#     #plt.plot(templates[unit].T)
#     plt.plot(wfs_array[unit].mean(0))

#     ax = plt.subplot(132)
#     #spikes = spikes[idx]
#     spikes = times[unit]
#     print ("spikes:", spikes.shape, "ptps: ", ptps[unit].shape)
#     plt.scatter(spikes/30000., ptps[unit])

#     ax=plt.subplot(133)
#     max_chan = wfs_array[unit].mean(0).ptp(0).argmax(0)#templates[unit].ptp(0).argmax(0)
#     print (wfs_array[unit].shape)

#     cmap = cm.get_cmap('viridis',wfs_array[unit].shape[0])
#     clrs = cmap(np.arange(wfs_array[unit].shape[0]))
#     print (clrs)
#     for k in range(0,wfs_array[unit].shape[0],10):
#         plt.plot(wfs_array[unit][k].T,c=cmap(k))


# plt.show()



spikes: (8210,) ptps:  (8210,)
(8210, 101)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]


In [11]:



#fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/standardized.bin'
#data_type = 'float32'
fname = '/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin'
data_type = 'int16'

n_channels = 60
n_times = 151

#spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
#spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output_original/spike_train.npy')
spike_train_gt = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')
    
    
#units = [199]
templates = []
wfs_array = []

# yass:  19 , gt:  561  ptp:  62.675797
# yass:  25 , gt:  235  ptp:  48.744278
# yass:  100 , gt:  541  ptp:  49.390488
# yass:  138 , gt:  516  ptp:  68.41851
# yass:  165 , gt:  480  ptp:  52.412918
# yass:  187 , gt:  502  ptp:  55.726055
            
unit_yass = 199
unit_gt = 217+1

# load yass waveforms
idx = np.where(spike_train_yass[:,1]==unit_yass)[0]
spikes = np.int32(spike_train_yass[idx,0])-n_times//2
wfs_yass = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_yass.shape)

# load 
idx = np.where(spike_train_gt[:,1]==unit_gt)[0]
spikes = np.int32(spike_train_gt[idx,0])-n_times//2+19
wfs_gt = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type)
print (wfs_gt.shape)    

(2873, 151, 60)
(5544, 151, 60)


In [65]:
template_yass = wfs_yass.mean(0)
print (template_yass.shape)
max_chans = template_yass.ptp(0)
idx = np.argsort(max_chans)[::-1]
print (idx)

for k in range(10):
    ax=plt.subplot(2,5,k+1)
    print (idx[k],wfs_yass.mean(0).shape)
    plt.plot(wfs_yass.mean(0)[:,idx[k]],c='red')
    plt.plot(wfs_gt.mean(0)[:,idx[k]],c='black')
    plt.xticks([])
    plt.yticks([])
    
plt.suptitle("Red: yass;  black: ground truth")
plt.show()

(151, 60)
[32 30 34 33 31 35 36 28 29 37 26 27 38 22 24 23 25 39 20 21 19 18 40 17
 16 41 14 15 12 42 13 43 10 11 44  8  9 45  6  7 46 47  4  5 48 49  2  3
  0  1 50 51 52 53 55 54 57 56 58 59]
32 (151, 60)
30 (151, 60)
34 (151, 60)
33 (151, 60)
31 (151, 60)
35 (151, 60)
36 (151, 60)
28 (151, 60)
29 (151, 60)
37 (151, 60)


In [69]:
max_chan = wfs_yass.mean(0).ptp(0).argmax(0)

plt.plot(wfs_yass[:,:,max_chan].T,c='red', alpha=.01)
plt.plot(wfs_gt[:,:,max_chan].T,c='black', alpha=.01)
plt.show()

In [75]:
max_chan = templates[0].ptp(0).argmax(0)
plt.plot(wfs[:,:,max_chan].T, c= 'black', alpha=.1)
plt.show()

In [38]:
plt.plot(templates[0])
plt.show()

In [42]:
data = np.fromfile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data.bin','int16')
print (data.shape)

(1152000000,)


In [44]:
data2D = data.reshape(60,-1).T
for k in range(60):
    plt.plot(data2D[:10000, k]+k*100,c='black')
plt.show()

In [45]:
data2D.ravel().tofile('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/data_correct_order.bin')

In [7]:
spike_train_yass = np.load('/media/cat/4TBSSD/data/spikingforest/synth_bionet_static/static_8x_A_2A/output/spike_train.npy')
print (spike_train_yass.shape)

(1081156, 2)


In [72]:
import csv

with open('employee_birthday.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')
    
print (data)